In [1]:
!pip install sqlalchemy psycopg2-binary
!pip install boto3
import boto3
import getpass
import pandas as pd
from sqlalchemy import create_engine

## Load the data

In [2]:
url = input("Enter your DB host (e.g., retaildb.xxxxxx.rds.amazonaws.com): ")
df = pd.read_csv(url)
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


## Summarize the data

In [3]:
df.info()
df.describe(include='all')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


Output:
- Date is a string → should convert to datetime
- Product Category, Gender, and Customer ID are categorical
- Total Amount = Quantity × Price per Unit (we can double-check this)

## Transforming the data

In [4]:
# 1. Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

# 2. Extract Year, Month, Day from 'Date'
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# 3. Add a 'Revenue Verified' column to check math
df['Revenue Verified'] = df['Quantity'] * df['Price per Unit'] == df['Total Amount']

# 4. Check if all revenues match
print("Revenue match for all rows:", df['Revenue Verified'].all())

# 5. Drop the verification column (optional)
df.drop(columns='Revenue Verified', inplace=True)

# Preview result
df.head()

Revenue match for all rows: True


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Year,Month,Day
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,2023,11,24
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,2023,2,27
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,2023,1,13
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,2023,5,21
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,2023,5,6


In [8]:
# Replace with your actual values
host = input("Enter your DB host (e.g., retaildb.xxxxxx.rds.amazonaws.com): ")
port = '5432'
database = 'postgres'
username = 'postgres'
password = getpass.getpass("Enter your DB password:")

# Create SQLAlchemy engine
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# Upload your DataFrame to PostgreSQL
df.to_sql('retail_sales', engine, if_exists='replace', index=False)

# Test query
pd.read_sql('SELECT * FROM retail_sales LIMIT 5', engine)

Enter your DB host (e.g., retaildb.xxxxxx.rds.amazonaws.com): retaildb.c3i8mag428vg.us-east-2.rds.amazonaws.com
Enter your DB password:··········


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Year,Month,Day
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,2023,11,24
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,2023,2,27
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,2023,1,13
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,2023,5,21
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,2023,5,6


In [9]:
pd.read_sql('SELECT COUNT(*) FROM retail_sales', engine)

,count
0,1000
